# Intro to LLMs (LLM-delen) — ryddige notater + kode

Dette er en “ryggmargs”-intro til **Large Language Models** (LLMs):

- hva de er
- hva de kan gjøre
- hvordan de funker (Transformer)
- hvordan du bruker dem i egne prosjekter (OpenAI / LangChain / Hugging Face)
- hvorfor dette er relevant for bachelor (sikkerhet, reproducibility)

> GitHub-tips: clear output før du pusher.


## 1) Hva er en LLM?

En **Large Language Model** er en språkmodell (ofte Transformer-basert) som er trent på enorme mengder tekst for å lære mønstre i språk.

### Kortversjon

- **Input**: tekst → tokeniseres
- **Modell**: lærer statistiske/semantiske mønstre og kontekst
- **Output**: genererer tekst (decoder-only, f.eks. GPT) eller lager representasjoner/klassifisering (encoder, f.eks. BERT)

### Typiske egenskaper

- **general purpose**: kan brukes til mange oppgaver
- **pre-trained** på store datasett, og kan så **fine-tunes** på spesifikke oppgaver
- kan fungere i **zero-shot** og **few-shot** settinger (prompting)


## 2) Hva kan LLMs brukes til?

Eksempler (du nevnte mange av disse):

- **tekstgenerering**: essays, e-post, poesi
- **oversettelse**: språk → språk
- **spørsmål/svar**: FAQ, dokument-QA
- **chatbots**: kundeservice, intern assistent
- **koding**: forslag, debugging, refactoring
- **analyse**: oppsummering, keyword extraction, klassifisering
- **domene**: finance, jus, helse (men krever ekstra kontroll/validering)

> Viktig i “ekte systemer”: kombiner ofte LLM med verktøy (retrieval, databaser, regler) for sporbarhet og sikkerhet.


## 3) Hvor store er LLMs (parametre) — og hvorfor betyr det noe?

- **Parametre** = “vekter” i nettverket (millioner → milliarder)
- Grovt: flere parametre + mer data kan gi bedre språkforståelse

Men:

- det er ikke *bare* størrelse (data-kvalitet, arkitektur, trening, alignment, osv.)
- større modeller kan være dyrere, tregere, og mer krevende å sikre

### LLM-trening i to steg

- **Pre-training**: lærer generelle språk-mønstre på store korpus (bøker, web, osv.)
- **Fine-tuning / instruction tuning**: gjør modellen mer spesifikk (oppgave, stil, sikkerhet)

### Prompting

- **zero-shot**: bare instruksjon
- **few-shot**: gir 1–N eksempler i prompten


## 4) Hvordan fungerer de? (Transformer – "Attention is All You Need")

### A) Tokenisering → embeddings

Tekst blir gjort om til **tokens** (subwords), og hvert token blir en vektor (**embedding**).

### B) Self-attention

Modellen regner ut hvilke tokens som er viktigst for hverandre (kontekst).

- **Multi-head attention** lar modellen fange flere typer relasjoner samtidig.

### C) Arkitekturtyper

- **Encoder-only** (BERT/RoBERTa/DistilBERT): gode på forståelse/klassifisering/QA (extractive)
- **Decoder-only** (GPT): gode på generering/chat
- **Encoder–decoder** (T5/BART): ofte brukt til oversettelse/summarization

Huske-regel:

- Encoder = “les/forstå”
- Decoder = “skriv/fortsett”


## 5) GPT, BERT, XLNet, LLaMA — hva er forskjellen?

### GPT (decoder-only)

- genererer tekst token-for-token
- brukes ofte til chat, generering, “reasoning” og tool-use

### BERT (encoder-only)

- trener på Masked Language Modeling (MLM)
- veldig sterk på klassifisering, NER, semantisk søk, extractive QA

### XLNet

- alternativ pretraining som kan gi gode resultater på noen tasks
- ofte brukt i klassifisering (historisk)

### LLaMA (familie av åpne modeller)

- open-weight modeller (varierer per versjon/lisens)
- ofte brukt lokalt / i egen infrastruktur

> Bachelor-relevans: modelltype påvirker angrepsflate (tokenisering, special tokens, prompt-format, osv.).


## 6) Hugging Face: lagring og lasting av modell (Bachelor-notater)

### Hvorfor lagre både modell og tokenizer?

I LLM-sikkerhet/backdoor/defense er **reproduserbarhet** alt.

> En modell uten riktig tokenizer er i praksis en annen modell.

### Hva lagres?

- Tokenizer: `tokenizer.json`, vocab/merges/sentencepiece, special tokens, mapping
- Modell: `pytorch_model.bin` + `config.json` (+ ev. `safetensors`)

### Typisk workflow

1. Evaluer baseline (CACC/ASR)
2. Lagre checkpoint
3. Kjør defense
4. Lagre ny checkpoint
5. Sammenlign før/etter


In [ ]:
# Hugging Face: save_pretrained / from_pretrained (eksempel)
# (Krever: transformers)

# from transformers import AutoTokenizer, AutoModelForSequenceClassification
#
# model_name = "distilbert-base-uncased"
# save_dir = "models/distilbert-base-uncased"
#
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSequenceClassification.from_pretrained(model_name)
#
# tokenizer.save_pretrained(save_dir)
# model.save_pretrained(save_dir)
#
# # reload
# tok2 = AutoTokenizer.from_pretrained(save_dir)
# model2 = AutoModelForSequenceClassification.from_pretrained(save_dir)
#
# print("Saved and reloaded OK")


## 7) Special tokens (viktig for både NLP og sikkerhet)

Special tokens gir modellen struktur (start/slutt, separator, padding, osv.).

- BERT: `[CLS]`, `[SEP]`, `[PAD]`, `[MASK]`
- RoBERTa: `<s>`, `</s>` (varierer)
- GPT: ofte `<bos>`, `<eos>` (varierer)

**Hvorfor det betyr noe i bachelor:**

- triggere kan være token-spesifikke
- padding/truncation kan skjule eller forsterke triggere
- endrer du tokenizer, endrer du angrepsflaten


In [ ]:
# Se special tokens i en tokenizer (eksempel)
# (Krever: transformers)

# from transformers import AutoTokenizer
# tok = AutoTokenizer.from_pretrained("bert-base-uncased")
# print(tok.special_tokens_map)
# print("PAD id:", tok.pad_token_id)
# print("CLS id:", tok.cls_token_id)
# print("SEP id:", tok.sep_token_id)


## 8) Fra LLM til ditt prosjekt: en praktisk mal

### A) Ren LLM (uten retrieval)

Bra for:

- ideer, tekst, forklaringer
- små kodehjelp

Ulempe:

- kan hallusinere
- ikke oppdatert på dine dokumenter

### B) RAG (Retrieval-Augmented Generation)

Når du vil svare med *dine* dokumenter:

1. last dokumenter
2. chunk/splitt
3. embeddings
4. lagre i vector store
5. retrieval + LLM-svar

### C) Logging + eval

- logg prompts, modell, temperatur, max tokens
- evaluer output (kvalitet, sikkerhet, ASR/CACC hvis backdoor)


## 9) OpenAI API (eksempel) — *bruk env vars*

Du hadde et eksempel med `openai==0.28`. Det funker, men moderne SDK-er har litt annen stil.

**Best practice (GitHub):**

- legg `OPENAI_API_KEY` i `.env`
- last med `%dotenv` eller `python-dotenv`
- aldri commit `.env`


In [ ]:
# OpenAI (legacy-style, omtrent som openai==0.28)
# NB: Kjør bare hvis du har openai-pakken installert og OPENAI_API_KEY satt.

# import os
# import openai
#
# openai.api_key = os.getenv("OPENAI_API_KEY")
#
# def generate_text(prompt: str, max_tokens: int = 50, temperature: float = 0.7) -> str:
#     resp = openai.Completion.create(
#         engine="davinci-002",
#         prompt=prompt,
#         max_tokens=max_tokens,
#         temperature=temperature,
#     )
#     return resp.choices[0].text.strip()
#
# print(generate_text("Once upon a time", max_tokens=30, temperature=0.0))


## 10) Mini-del: QA (BERT) + klassifisering (XLNet) — rapportvennlig

Dette er to veldig “klassiske” NLP-demoer som også er fine å beskrive i bachelor:

- **Extractive QA**: modellen velger start/slutt-posisjon i kontekst
- **Klassifisering**: modellen predikerer label (følelse, spam, topic, osv.)

Hvis du vil, kan jeg lage disse som egne notebooks i `nlp/` senere.


## 11) LLM-sikkerhet: angrep vs defense (rammeverk)

### Attack surface (hvor angrep kan skje)

- **data-nivå**: poisoning/backdoor i treningsdata
- **modell-nivå**: kompromitterte checkpoints (supply chain)
- **prompt/kontekst**: prompt injection, RAG-dokumenter med instruksjoner

### Defense (typisk pipeline)

- data filtering/anomali
- post-training defenses (pruning/unlearning/merge)
- inference defenses (sanitization, prompt firewall)

### Evaluering

- **CACC** (clean accuracy) og **ASR** (attack success rate)
- logg alt: modellnavn, seed, hyperparametre, triggere


## 12) Kurs-repo / miljø (fra notatene dine)

Du limte inn info fra et repo med "Introduction to LLMs".

### Eksempel på pakker (kurs-miljø)

*(Dette er “pinne”-versjoner fra kurset; du kan bruke nyere i eget prosjekt.)*

- python 3.11
- `transformers`, `torch`, `datasets`, `evaluate`, `accelerate`
- `langchain`, `faiss-cpu`, `tiktoken`

### Conda eksempel (rettet kommando-format)

```bash
conda create --name llms_course_env python=3.11
conda activate llms_course_env
pip install ipykernel jupyterlab notebook
python -m ipykernel install --user --name=llms_course_env --display-name "llms_course_env"
```

> Hvis du heller bruker `venv` (som i `03-setting-up-the-env.ipynb`), er det helt fint.
